# LSTM on mnist

In [1]:
# -*- coding:utf-8 -*-
import tensorflow as tf
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import np_utils

D:\study\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 载入数据

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

tf.set_random_seed(1)
np.random.seed(1)

# Hyper Parameters
batch_size = 300
time_steps = 28          # rnn time step / image height
input_size = 28         # rnn input size / image width
lr = 0.005               # learning rate
num_class = 10

## Model

In [3]:
image = tf.placeholder(tf.float32, [batch_size, 28, 28])
y = tf.placeholder(tf.int32, [None, num_class])

lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=64)
outputs, (h_m, h_b) = tf.nn.dynamic_rnn(
    lstm_cell,                   # cell you have chosen
    image,                      # input
    initial_state=None,         # the initial hidden state
    dtype=tf.float32,           # must given if set initial_state = None
    time_major=False,           # False: (batch, time step, input); True: (time step, batch, input)
)

In [4]:
pred = tf.layers.dense(outputs[:, -1, :], num_class)
loss = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=pred)
train_op = tf.train.AdamOptimizer(lr).minimize(loss)
a = tf.equal(tf.argmax(y, axis=1), tf.argmax(pred, axis=1))
accuracy = tf.reduce_mean(tf.cast(a, tf.float32))

In [5]:
class Batch(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self._point = 0
        self._dataSize = X.shape[0]   #样本总数

    def next_batch(self, batch_size, shuffle=True):
        if batch_size > self._dataSize:
            raise RangeError

        if shuffle == True:
            index = np.random.randint(0, self._dataSize, batch_size, dtype=int)
            return self.X[index], self.y[index]
        
        elif shuffle != True:
            pass

In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
#     writer = tf.summary.FileWriter(r"D:\Repositories\Machine-Learning\jupyter notebook\tensofboard", sess.graph)
    train_round = 1000
    batch = Batch(X_train, y_train)
    
    for i in range(train_round):
        train_X, train_y = batch.next_batch(batch_size)
        train_y = np_utils.to_categorical(train_y, num_class)
        _, _loss, _accuracy= sess.run([train_op, loss, accuracy], feed_dict={image: train_X, y: train_y})
        if i % 50 == 0:
            print('loss: %.4f  accuracy: %.2f' % (_loss, _accuracy))

loss: 2.4071  accuracy: 0.07
loss: 0.7468  accuracy: 0.74
loss: 0.5179  accuracy: 0.82
loss: 0.4527  accuracy: 0.86
loss: 0.4488  accuracy: 0.85
loss: 0.4978  accuracy: 0.83
loss: 0.3667  accuracy: 0.88
loss: 0.3336  accuracy: 0.89
loss: 0.2879  accuracy: 0.89
loss: 0.3017  accuracy: 0.90
loss: 0.2790  accuracy: 0.91
loss: 0.3678  accuracy: 0.88
loss: 0.2672  accuracy: 0.92
loss: 0.2765  accuracy: 0.90
loss: 0.2524  accuracy: 0.90
loss: 0.3256  accuracy: 0.90
loss: 0.2834  accuracy: 0.91
loss: 0.2235  accuracy: 0.94
loss: 0.3001  accuracy: 0.90
loss: 0.2183  accuracy: 0.92


正确率达 90%